# 행안부 연령별 인구현황( https://jumin.mois.go.kr/index.jsp )

In [ ]:
!dir *.csv

dir: cannot access '*.csv': No such file or directory


In [ ]:
!cp *.csv /content/drive/MyDrive/

cp: cannot stat '*.csv': No such file or directory


In [ ]:
import pandas as pd

filename="/content/drive/MyDrive/202112_202112_연령별인구현황_월간.csv"
jumin = pd.read_csv(filename, encoding="euc-kr", thousands=",")
jumin

In [ ]:
jumin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3861 entries, 0 to 3860
Columns: 104 entries, 행정구역 to 2021년12월_계_100세 이상
dtypes: int64(103), object(1)
memory usage: 3.1+ MB


In [ ]:
# 행정구역의 법정동코드와 시군구를 분리
jumin["행정구역"].str.split("(", expand=True) # 분리한것을 칼럼으로 생성, 반환

In [ ]:
# 행정구역의 법정동코드와 시군구를 분리.index기준 가로 합체
jumin = jumin.join( 
    jumin["행정구역"].str.split("(", expand=True) 
)
jumin.head(3)

In [ ]:
# 맨우측 괄호 삭제
jumin[1] = jumin[1].str.replace(")","")
jumin.head(1)

In [ ]:
# 칼럼명 수정
jumin.rename(
    {0:"시군구", 1:"법정동코드"}, 
    axis=1, 
    inplace=True )
# 독산동 인구현황 추출
# "시군구"번 칼럼값이 "독산동"으로 끝나는 행 추출
jumin[  jumin["시군구"].str.endswith("독산제1동")  ]  # xxx로 끝나는 문자열
# jumin[  jumin["시군구"].str.contains("독산")  ]  # xx를 포함하는 문자열
# jumin[  jumin["시군구"].str.startswith("제주")  ]  # xx로 시작하는 문자열

In [ ]:
# 독산제1동정보 추출
독산제1동 = jumin[  jumin["시군구"].str.endswith("독산제1동")  ]  # xxx로 끝나는 문자열
독산제1동

In [ ]:
# 코랩 한글 적용
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf
# 코랩 한글 적용 후 런타임 재시작
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic')
plt.rc('axes', unicode_minus=False)

In [ ]:
# 독산제1동의 연령별 인구분포 추출하여 선그래프로 그리기
# 선그래프는 칼럼당 1줄이기에 행을  추출하여 행과 열의 위치 바꾸가
# "2021년12월_계_0세" ~ "2021년12월_계_100세 이상" 추출
# 독산제1동.T
독산제1동.loc[ : , "2021년12월_계_0세":"2021년12월_계_100세 이상" ].T.plot(figsize=(16, 3))

In [ ]:
# 독산제1동과 가장 비슷한 인구 분포를 가진 동을 찾으세요.
# 독산제1동의 연령별 인구수와 다른동의 연령별 인구수의 차이의 제곱의 합이 가장 작은 동을 찾기
# 1. 0세~ 10세이상만 추출
# 2. 독산제1동 추출
# 3. 다른동추출, 반복, 연령별 차이 제곱의 합 계산
# 4. 계산한 값의 최소값을 저장 

# 동별 0세~100세이상 칼럼만 추출
전체연령 = jumin.loc[ : , "2021년12월_계_0세":"2021년12월_계_100세 이상"]
전체연령.info()

In [ ]:
독산제1동연령 = 독산제1동.loc[ :, "2021년12월_계_0세":"2021년12월_계_100세 이상"]
독산제1동연령

In [ ]:
# 계산방법 예제
# 독산제1동에서 전체연령의 2번 행의 차이를 계산
독산제1동연령 - 전체연령.loc[2, :]

In [ ]:
# # 독산제1동에서 전체연령의 2번 행의 차이의 제곱을 계산
(독산제1동연령 - 전체연령.loc[2, :] )**2

In [ ]:
# # 독산제1동에서 전체연령의 2번 행의 차이의 제곱을 계산하여 변수에 저장
연령별차이제곱 = (독산제1동연령 - 전체연령.loc[2, :] )**2
연령별차이제곱 # DF

In [ ]:
# # 독산제1동에서 전체연령의 2번 행의 차이의 제곱을 계산한것의 총합
연령별차이제곱.loc[296].sum() # 연령별차이제곱의 전체합 -> 모든 행에 대해 계산, 최소값 찾기(반복문)

In [ ]:
# 한줄로 계산
((독산제1동연령 - 전체연령.loc[2, :] )**2).loc[296].sum() 

In [ ]:
# 동별 0세~100세이상 칼럼만 추출
# 기준동 제외 조건
전체연령 = jumin.loc[ 
                 ~jumin["시군구"].str.endswith("독산제1동")  , 
                 "2021년12월_계_0세":"2021년12월_계_100세 이상" 
                 ]
#전체연령의 index는 순차번호

최소연령별차이제곱의합 = 999999999999999
최소연령별차이제곱의합idx = -1

# 전체연령의 모든 행을 반복
for idx in 전체연령.index:
  #idx번째 행 추출
  연령별차이제곱의합 = ((독산제1동연령 - 전체연령.loc[idx, :] )**2).loc[296].sum() 
  if 연령별차이제곱의합 < 최소연령별차이제곱의합:
    최소연령별차이제곱의합 = 연령별차이제곱의합
    최소연령별차이제곱의합idx = idx

print("최소연령별차이제곱의합",최소연령별차이제곱의합)
print("최소연령별차이제곱의합idx",최소연령별차이제곱의합idx)
print("최소연령별차이의 동이름",jumin.loc[최소연령별차이제곱의합idx, "시군구"])

In [ ]:
# 독산제1동, 경기도 수원시 팔달구 인계동의 연령별 인구분포 선그래프 출력
jumin.loc[ 
          [296,1312 ], 
          "2021년12월_계_0세":"2021년12월_계_100세 이상"].T.plot(figsize=(16,4))

In [ ]:
# 독산제1동과 비슷한 10개동 검색
# 최소연령별차이제곱의합을 새로운 칼럼에 저장
# 최소연령별차이제곱의합을 리스트에 저장, 리스트를 칼럼에 저장

전체연령 = jumin.loc[ 
                 :  , 
                 "2021년12월_계_0세":"2021년12월_계_100세 이상" 
                 ]
#전체연령의 index는 순차번호

연령별차이제곱의합_리스트 = [] 

# 전체연령의 모든 행을 반복
for idx in 전체연령.index:
  #idx번째 행 추출
  연령별차이제곱의합 = ((독산제1동연령 - 전체연령.loc[idx, :] )**2).loc[296].sum() 
  연령별차이제곱의합_리스트.append( 연령별차이제곱의합 )

jumin["연령별차이제곱의합"] = 연령별차이제곱의합_리스트


In [ ]:
jumin.sort_values(by="연령별차이제곱의합").head().loc[ :,"2021년12월_계_0세":"2021년12월_계_100세 이상"].T.plot(figsize=(20,4))

In [ ]:
jumin.set_index("시군구").sort_values(by="연령별차이제곱의합").tail().loc[ :,"2021년12월_계_0세":"2021년12월_계_100세 이상"].T.plot(figsize=(20,4))

In [ ]:
#행정구역의 끝글자의 종류는?
jumin["시군구"].str.strip().str.get(-1).unique()

In [ ]:
# 공백이 두개 이상 있는 것. strip() => 앞뒤공백 글자 지우기
jumin[ jumin["시군구"].str.strip().str.count(" ") >= 2 ] 

In [ ]:
# 독산제1동 연령별 인구분포와 차이가 가장 큰 동 5개의 그래프
jumin[ 
      jumin["시군구"].str.strip().str.count(" ") >= 2  # 행선택 조건. 1,2레벨 행정구역 제외(서울시, 서울시 강동구 이런 패턴 제외)
    ].set_index("시군구").sort_values(by="연령별차이제곱의합").tail().loc[
                                                              : ,
                                                              "2021년12월_계_0세":"2021년12월_계_100세 이상"].T.plot(figsize=(20,4))